In [1]:
import nltk
import pandas as pd
import sys
import os
import io
import operator

from data_loader import DataLoader

from nltk.tokenize import word_tokenize ,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.cluster.util import cosine_distance
from nltk import sent_tokenize, word_tokenize, PorterStemmer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from nltk import pos_tag

import math

import numpy

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

import networkx as nx

import re

import networkx as nx

from collections import defaultdict
import string

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import copy
import itertools

import pke

from  itertools import chain

import random
random.seed(0)

import time

In [2]:
loader = DataLoader(DataLoader.data_path2)
table = loader.load_table()
# table.head()

C:\Users\lijia\Desktop\MSAI\Sem A\AI6122\Assignment1\Hearts-of-Iron\data_loader.py:14: FutureWarning: Starting with pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only.
  self.table = pd.read_json(data_path, 'records', lines = True);


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
85,A1C0C9IJO2HS03,B00000016W,A Customer,"[1, 1]",Brian Wilson was so blown away after hearing R...,5,Holy jeez this is good!,1044489600,"02 6, 2003"
86,A22N9H8V0RYQR3,B00000016W,A fair and Balanced Rater,"[4, 29]",I never understood what's the BIG deal behind ...,1,Overrated HYPE,1125273600,"08 29, 2005"
87,A1E110L9ZKX6FD,B00000016W,All Powerful Wizard Of Oz,"[9, 10]",For those that say they &quot;don't get it&quo...,5,"Lets Break it down, coming from a musican.....",1078185600,"03 2, 2004"
88,A3HU0B9XUEVHIM,B00000016W,"Andre S. Grindle ""Andre' Grindle""","[0, 0]",Brian Wilson once said of this album he wanted...,5,It Was Uniquely Wonderful Than. And It Still R...,1321056000,"11 12, 2011"
89,A252HNVAZENVNE,B00000016W,"Andre' S Grindle ""Funk Meister""","[1, 3]","You know,even with classic radio hits like &qu...",5,A Classic All The Way Through!,1060732800,"08 13, 2003"


In [3]:
#Data Preprocess
#Get stopwords
nltk_stopwords = stopwords.words('english')
nltk_stopwords.append('\n')
spacy_stopwords = list(STOP_WORDS)
merged_sw = nltk_stopwords + list(set(spacy_stopwords) - set(nltk_stopwords))

# print(merged_sw)

nltk:  180
spacy:  326
merge:  383


In [4]:
#Create the review text dictionary
reviewText = {}
for i in table.index:
    if table['asin'][i] in reviewText:
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)
    else:
        reviewText[table['asin'][i]] = []
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)

In [5]:
def data_preprocess(original_review, stop_words):
    # to lowercase
    review = original_review.lower()
    # remove punctuation
    review = re.sub(r'[^\w\s]', ' ', review).strip()

    lemmatizer = WordNetLemmatizer()
    review_words = review.split(' ')
    review_words_tag = nltk.pos_tag(review_words)
    processed_review = ""
    # remove stop words&lemma
    for word, tag in review_words_tag:
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma_word = word
        else:
            lemma_word = lemmatizer.lemmatize(word, wntag)
        if lemma_word not in stop_words:
            processed_review = processed_review + lemma_word + " "
    processed_review = " ".join(processed_review.split())
    processed_review = processed_review.strip()
    return processed_review



In [6]:
processed_reviewText = {}
for r in reviewText:
    processed_reviewText[r] = []
    for t in reviewText[r]:
        p = data_preprocess(t, merged_sw)
        processed_reviewText[r].append(p)
# print(processed_reviewText)

In [7]:
# collect frequency

def collect_frequency(processed_review):
    word_frequency = defaultdict(lambda:0)
    for v in processed_review.values():
        for i in v:
            tokens = nltk.word_tokenize(i)
            for token in tokens:
                word_frequency[token] += 1  
    return word_frequency

frequency = collect_frequency(processed_reviewText)

In [8]:
frequency = sorted(frequency.items(), key=lambda item: item[1],reverse=True)
# print(frequency)

In [9]:
sorted_processed_reviewText = sorted(processed_reviewText.items(), key=lambda item: len(item[1]),reverse=True)
# print(sorted_processed_reviewText[:3])

In [10]:
print(len(processed_reviewText[sorted_processed_reviewText[0][0]]))

3027


In [20]:
text = """"""
for i in reviewText[sorted_processed_reviewText[0][0]]:
    text+=i
# print(text)

In [21]:
# Baseline Models Cite from https://github.com/boudinfl/pke

In [22]:
start = time.time()
#TextRank
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input=text,
                        language='en',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)

# 4. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)

print(keyphrases)
end = time.time()
print(end - start)

[('a good rap album .', 0.019971753091184406), ('a good rap album', 0.01892312442541186), ('other commercial rap album', 0.017162462137877112), ('a real gangsta rap album', 0.016830597486591596), ('many good mainstream rap albums', 0.01578108772295227), ('a good album .', 0.015650519909826452), ('the previous album the last album', 0.015634677769476645), ('a good song 5/55.piggy bank', 0.014602359354647088), ('album iz good', 0.014434457187851154), ('a good album', 0.014172151244053906)]
4.067541122436523


In [23]:
start = time.time()
# 1. create a YAKE extractor.
extractor = pke.unsupervised.YAKE()

# 2. load the content of the document.
extractor.load_document(input=text,
                        language='en',
                        normalization=None,
                        stoplist=merged_sw)


# 3. select {1-3}-grams not containing punctuation marks and not
#    beginning/ending with a stopword as candidates.
extractor.candidate_selection(n=3)

# 4. weight the candidates using YAKE weighting scheme, a window (in
#    words) for computing left/right contexts can be specified.
window = 2
use_stems = False # use stems instead of words for weighting
extractor.candidate_weighting(window=window,
                              use_stems=use_stems)

# 5. get the 10-highest scored candidates as keyphrases.
#    redundant keyphrases are removed from the output using levenshtein
#    distance and a threshold.
threshold = 0.8
keyphrases = extractor.get_n_best(n=10, threshold=threshold)

print(keyphrases)

end = time.time()
print(end - start)

[('baltimore love thing', 7.136342516303575e-05), ('candy shop', 0.00015989046436451352), ('die tryin', 0.0002617178314869215), ('ski mask way', 0.00026577914464383776), ('die tonight', 0.0002671537525698024), ('baltimore love', 0.00028511253329892213), ('album', 0.00030649095792677894), ('cent', 0.00032256545009830615), ('love thing', 0.0003644474800517442), ('die', 0.00047382669454123176)]
7.242422580718994


In [24]:
start = time.time()
# 1. create a TfIdf extractor.
extractor = pke.unsupervised.TfIdf()

# 2. load the content of the document.
stoplist = list(string.punctuation)
stoplist += pke.lang.stopwords.get('en')
extractor.load_document(input=text,
                        language='en',
                        stoplist=merged_sw,
                        normalization=None)

# 3. select {1-3}-grams not containing punctuation marks as candidates.
extractor.candidate_selection(n=3)

# 4. weight the candidates using a `tf` x `idf`
df = pke.load_document_frequency_file(input_file='df.tsv.gz')
extractor.candidate_weighting(df=df)

# 5. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)

print(keyphrases)

end = time.time()
print(end - start)

[('album', 887.5790004038475), ('like', 664.0992878021644), ('song', 505.60303773004887), ('cent', 340.7669376550486), ('songs', 313.82257514278893), ('love', 275.7834751254812), ('beat', 269.4436251225966), ('die', 263.10377511971194), ('50 cent', 253.594000115385), ('music', 239.3293376088946)]
6.076050758361816


In [25]:
start = time.time()
# TopicRank
# initialize keyphrase extraction model, here TopicRank
extractor = pke.unsupervised.TopicRank()

# load text
extractor.load_document(input=text, language='en')

# keyphrase candidate selection, in the case of TopicRank: sequences of nouns
# and adjectives (i.e. `(Noun|Adj)*`)
extractor.candidate_selection()

# candidate weighting, in the case of TopicRank: using a random walk algorithm
extractor.candidate_weighting()

# N-best selection, keyphrases contains the 10 highest scored candidates as
# (keyphrase, score) tuples
keyphrases = extractor.get_n_best(n=10)

print(keyphrases)

end = time.time()
print(end - start)

[('album', 0.033753948362687455), ('song', 0.023967132116933014), ('beat', 0.014620312722206206), ('good', 0.013592454845410724), ('cent', 0.013138238057820358), ('track', 0.0109398281042443), ('game', 0.01057093102053484), ('good music', 0.009073734279705733), ('micsthe massacre', 0.008359861980728623), ('rich', 0.007891583946772711)]
68.7447738647461


In [26]:
# RAKE refers to https://github.com/fabianvf/python-rake

In [27]:
# RAKE Algorithm
def is_num(current_word):
    try:
        float(s) if '.' in s else int(s)
        return True
    except ValueError:
        return False
        
def split_words(text):
    splitter = re.compile(r'(?u)\W+')
    
    words = []
    for singleWord in splitter.split(text):
        currentWord = singleWord.strip().lower()
        
        if currentWord != '' and not is_num(currentWord):
            words.append(currentWord)
            
    return words


def build_regex(stopwords):
    sw_regex_list = []
    for word in stopwords:
        word_regex = r'\b' + word + r'(?![\w-])'
        sw_regex_list.append(word_regex)
        
    return re.compile('(?u)' + '|'.join(sw_regex_list), re.IGNORECASE)


def generate_keywords(sentences, stopWordPattern, minCharacters, maxWords):
    phrases = []
    for s in sentences:
        tmp = re.sub(stopWordPattern, '|', s.strip())
        ps = tmp.split("|")
        
        for phrase in ps:
            phrase = phrase.strip().lower()
            
            if phrase != '' and len(phrase) >= minCharacters and len(phrase.split()) <= maxWords:
                phrases.append(phrase)
                
    return phrases


def word_scores(phraseList):
    frequency = {}
    degree = {}
    for phrase in phraseList:
        wordList = split_words(phrase)
        wordListLen = len(wordList)
        wordListDegree = wordListLen - 1
        for word in wordList:
            frequency.setdefault(word, 0)
            frequency[word] += 1
            degree.setdefault(word, 0)
            degree[word] += wordListDegree
    for i in frequency:
        degree[i] = frequency[i] + degree[i]
    score = {}
    for i in frequency:
        score.setdefault(i, 0)
        score[i] = degree[i] / (frequency[i] * 1.0)
    return score


def generate_candidate_keyword_scores(phraseList, wordScore, minFrequency):
    ckScore = {}

    counts = defaultdict(int)
    for p in phraseList:
        counts[p] += 1

    for phrase in phraseList:
        if counts[phrase] >= minFrequency:
            ckScore.setdefault(phrase, 0)
            wordList = split_words(phrase)
            candidateScore = 0
            for word in wordList:
                candidateScore += wordScore[word]
            ckScore[phrase] = candidateScore
    return ckScore

def Rake(text, minCharNum, maxWordNum, minFrequency):
        sentences = re.compile(u'[.!?,;:\t\\\\"\\(\\)\\\'\u2019\u2013]|\\s\\-\\s').split(text)

        phrases = generate_keywords(sentences, build_regex(merged_sw), minCharNum, maxWordNum)

        wordScores = word_scores(phrases)

        ck = generate_candidate_keyword_scores(phrases, wordScores, minFrequency)

        sortedKeywords = sorted(ck.items(), key=operator.itemgetter(1), reverse=True)
        
        return sortedKeywords

In [28]:
start = time.time()

keywords = Rake(text, 2, 4, 2)
print(keywords[:10])

end = time.time()
print(end - start)

[('ski mask way 5/5', 14.479115796419167), ('3 1/2 stars', 10.963016643230908), ('4 1/2 stars', 10.61302421554652), ('song rating 3/10', 10.610885993321785), ('song rating 8/10', 10.302368220015278), ('song rating 9/10', 10.267736185383244), ('song rating 7/10', 10.151574569221628), ('50 comes pretty nice', 9.25118433530981), ('got shot 9 times', 9.113217646550979), ('outta control remix ft', 9.106313131313131)]
0.5256614685058594
